In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
# Creating training and testing dataset
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Training data
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

# Test Data
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

# Validation data
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

data_signature = infer_signature(train_x, train_y)

In [4]:
# Normalizing target label for Sparse Categorical Cross Entropy

train_y -= 3
val_y -= 3
test_y -= 3

train_y = train_y.astype(np.float32)
val_y = val_y.astype(np.float32)
test_y = test_y.astype(np.float32)


In [5]:
print(train_x.shape)
print(val_x.shape)
print(test_x.shape)

(3134, 11)
(784, 11)
(980, 11)


In [23]:
## Neural Net Model development

def train(params, epochs, train_x, train_y, val_x, val_y, test_x, test_y):

    ## Define Model Architecture

    # mean = np.mean(train_x, axis=0)
    # var = np.var(train_x, axis=0)

    norm_layer = keras.layers.Normalization()
    norm_layer.adapt(train_x)

    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        norm_layer,
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=params['lr']),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsoluteError()] 
    )


    # # Early stopping callback for best model retention
    # early_stop = keras.callbacks.EarlyStopping(
    #     monitor='val_loss', patience=10, restore_best_weights=True
    # )

    ## Train the ANN model with hyper paremetrs with MLFlow Tracking

    with mlflow.start_run(nested=True):
        # Fit the model
        model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=epochs, batch_size=32, verbose=0)


        # Predict and Evaluate Model
        y_pred = model.predict(test_x).flatten()
        rmse = np.sqrt(mean_squared_error(test_y, y_pred))
        mae = mean_absolute_error(test_y, y_pred)
        r2 = r2_score(test_y, y_pred)
        # eval_result = model.evaluate(test_x, test_y, batch_size=16, verbose=0)


        # Log hyperparameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("test_rmse", rmse)
        mlflow.log_metric("test_mae", mae)
        mlflow.log_metric("test_r2", r2)

        # log the model

        mlflow.tensorflow.log_model(model, "model", signature=data_signature)

        return {
            "loss": rmse,  # If using for hyperopt, minimize this
            "status": STATUS_OK,
            "model": model,
            "test_rmse": rmse,
            "test_mae": mae,
            "test_r2": r2
        }

In [24]:
def objective(params):
    result = train(
        params,
        epochs=30,
        train_x=train_x,
        train_y=train_y,
        val_x=val_x,
        val_y=val_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [26]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(5e-3))
}
mlflow.set_experiment("/wine-quality")
with mlflow.start_run(run_name="WineQuality_Hyperopt_Run"):
    # Conduct the hyperparameter search using Hperopt

    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals = 16,
        trials = trials,
        rstate=np.random.default_rng(42)
    )

    # Get best run/result
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best hyperparameters
    mlflow.log_params(best)

    # Log best metrics
    mlflow.log_metric("RMSE", best_run["test_rmse"])
    mlflow.log_metric("MAE", best_run["test_mae"])
    mlflow.log_metric("R2", best_run["test_r2"])

    # Log the best model
    mlflow.tensorflow.log_model(best_run['model'], "best_model", signature=data_signature)

    # Print out the best parameters and loss

    # Print out the best parameters and loss
    print(f"Best Parameters: {best}")
    print(f"Best RMSE: {best_run['test_rmse']}")
    print(f"Best MAE: {best_run['test_mae']}")
    print(f"Best R^2: {best_run['test_r2']}")

 1/31 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

  0%|          | 0/16 [00:06<?, ?trial/s, best loss=?]

2025/07/09 01:34:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpyo09igbc\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 



 1/31 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step                
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step        
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step       

  6%|▋         | 1/16 [00:17<02:53, 11.55s/trial, best loss: 0.6989593505859375]

2025/07/09 01:34:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpyzp7fhtm\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 



 1/31 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step                
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step        
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step       

 12%|█▎        | 2/16 [00:28<02:35, 11.12s/trial, best loss: 0.6989593505859375]

...

 1/31 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step                 
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step         

 94%|█████████▍| 15/16 [03:06<00:12, 12.08s/trial, best loss: 0.6868072748184204]

2025/07/09 01:37:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpon3_your\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 



100%|██████████| 16/16 [03:12<00:00, 12.00s/trial, best loss: 0.6868072748184204]


2025/07/09 01:37:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Sahil\AppData\Local\Temp\tmpkfrg83yl\model, flavor: tensorflow). Fall back to return ['tensorflow==2.16.1', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


Best Parameters: {'lr': 0.001282722267037808}
Best RMSE: 0.6868072748184204
Best MAE: 0.535742461681366
Best R^2: 0.39093534816198205
